In [2]:
import yfinance as yf
dat = yf.Ticker("MSFT")

In [ ]:
dat = yf.Ticker("MSFT")
dat.info
dat.calendar
dat.analyst_price_targets
dat.quarterly_income_stmt
dat.history(period='1mo')
dat.option_chain(dat.options[0]).calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,MSFT250502C00230000,2025-04-07 19:34:07+00:00,230.0,129.45,156.9,157.65,0.0,0.0,212.0,191,0.000010,True,REGULAR,USD
1,MSFT250502C00240000,2025-04-24 16:28:33+00:00,240.0,144.91,146.9,147.70,0.0,0.0,1.0,35,0.000010,True,REGULAR,USD
2,MSFT250502C00250000,2025-04-10 19:26:03+00:00,250.0,133.57,136.9,137.80,0.0,0.0,72.0,73,0.000010,True,REGULAR,USD
3,MSFT250502C00275000,2025-04-24 18:37:10+00:00,275.0,112.10,111.9,112.70,0.0,0.0,NaN,1,0.000010,True,REGULAR,USD
4,MSFT250502C00280000,2025-04-17 14:07:33+00:00,280.0,91.32,106.8,107.65,0.0,0.0,70.0,70,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,MSFT250502C00470000,2025-04-17 17:33:56+00:00,470.0,0.05,0.0,0.02,0.0,0.0,1.0,9,0.546880,False,REGULAR,USD
65,MSFT250502C00475000,2025-03-28 16:59:51+00:00,475.0,0.06,0.0,0.03,0.0,0.0,2.0,2,0.593754,False,REGULAR,USD
66,MSFT250502C00500000,2025-04-16 19:40:51+00:00,500.0,0.01,0.0,0.01,0.0,0.0,1.0,133,0.656253,False,REGULAR,USD
67,MSFT250502C00510000,2025-04-01 19:49:18+00:00,510.0,0.16,0.0,0.03,0.0,0.0,1.0,0,0.773440,False,REGULAR,USD


In [3]:
from data_fetch import get_price
start = '2024-01-01'
end = '2025-01-01'
prices = get_price(['AAPL', 'MSFT'], start, end)
prices

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  2 of 2 completed


Price            Close                    High                     Low  \
Ticker            AAPL        MSFT        AAPL        MSFT        AAPL   
Date                                                                     
2024-01-02  184.532089  367.380615  187.315382  372.363289  182.792533   
2024-01-03  183.150391  367.113159  184.770668  369.748136  182.335277   
2024-01-04  180.824356  364.478149  181.997307  369.589604  179.800504   
2024-01-05  180.098709  364.289948  181.669281  368.559394  179.094742   
2024-01-08  184.452560  371.164673  184.492330  371.669884  180.416793   
...                ...         ...         ...         ...         ...   
2024-12-24  257.916443  438.450836  257.926411  438.720315  255.009620   
2024-12-26  258.735504  437.233276  259.814335  440.057630  257.347047   
2024-12-27  255.309296  429.668457  258.415896  434.349074  252.782075   
2024-12-30  251.923019  423.979858  253.221595  426.694417  250.474615   
2024-12-31  250.144974  420.656525  253.001833  425.876070  249.156056   

Price                         Open                Volume            
Ticker            MSFT        AAPL        MSFT      AAPL      MSFT  
Date                                                                
2024-01-02  363.319185  186.033072  370.342474  82488700  25258600  
2024-01-03  365.042827  183.120571  365.538123  58414500  23083500  
2024-01-04  363.715405  181.062914  367.182474  71983600  20901500  
2024-01-05  363.051708  180.903888  365.498470  62303300  20987000  
2024-01-08  365.538121  181.003268  365.825371  59144500  23134000  
...                ...         ...         ...       ...       ...  
2024-12-24  433.321138  255.209412  433.780209  23234700   7164500  
2024-12-26  435.756258  257.906429  438.201337  27237100   8194200  
2024-12-27  425.496829  257.546826  433.730320  42355300  18117700  
2024-12-30  421.055729  251.952985  425.207408  35557500  13158700  
2024-12-31  419.818209  252.162760  425.247326  39480700  13246500  

[252 rows x 10 columns]

In [ ]:
#setting pandas to display floats with 2 decimal places
import pandas as pd

# Set pandas to display floats with 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

In [6]:
prices['Close']

Ticker,AAPL,MSFT
Date,,
2024-01-02,184.53,367.38
2024-01-03,183.15,367.11
2024-01-04,180.82,364.48
2024-01-05,180.10,364.29
2024-01-08,184.45,371.16
...,...,...
2024-12-24,257.92,438.45
2024-12-26,258.74,437.23
2024-12-27,255.31,429.67


In [51]:
from data_fetch import get_price
from portfolio import Portfolio
from plotter import plot_prices

shares = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'NFLX']

portfolio = Portfolio()
for symbol in shares:
    portfolio.add_asset(symbol, 1)



start = '2024-01-01'
end = '2025-01-01'

prices_df = get_price(shares, start, end)
prices_close_df = prices_df['Close']

print("Portfolio Value:", portfolio.get_portfolio_value(prices_close_df))

[*********************100%***********************]  7 of 7 completed

5 Failed downloads:
['GOOG', 'NFLX', 'AMZN', 'TSLA', 'NVDA']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Portfolio Value: Date
2024-01-02   NaN
2024-01-03   NaN
2024-01-04   NaN
2024-01-05   NaN
2024-01-08   NaN
              ..
2024-12-24   NaN
2024-12-26   NaN
2024-12-27   NaN
2024-12-30   NaN
2024-12-31   NaN
Name: AAPL, Length: 252, dtype: float64
